### Notes

* changed shape[0] from 1 to 4

In [1]:
%matplotlib inline

In [2]:
import sys
import numpy as np
import tensorflow as tf

from batcher import Batcher
from metrics import acc012

In [3]:
X_train = np.load('../../data/bin/train/deal.npy')
y_train = np.load('../../data/bin/train/tricks_notrump.npy')

X_val = np.load('../../data/bin/val/deal.npy')
y_val = np.load('../../data/bin/val/tricks_notrump.npy')

n_examples = X_train.shape[0]

n_h = X_train.shape[1]
n_w = X_train.shape[2]
n_c = X_train.shape[3]

batch_size = 64
n_iterations = 2000000
display_step = 100000

learning_rate = 0.001

n_hidden_units = 128

l2_reg = 0.05

In [4]:
X_train.shape, X_val.shape, y_train.shape, y_val.shape

((10800000, 4, 13, 4), (100000, 4, 13, 4), (10800000, 1), (100000, 1))

In [5]:
X = tf.placeholder(tf.float32, shape=[None, n_h, n_w, n_c])
Y = tf.placeholder(tf.float32, shape=[1, None])
strides = [1,1,1,1]

conv1_w = tf.get_variable('c1w', shape=[4, 4, 4, 32], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv1_z = tf.nn.conv2d(X, filter=conv1_w, strides=strides, padding='SAME')
conv1_a = tf.nn.relu(conv1_z)

conv2_w = tf.get_variable('c2w', shape=[4, 4, 32, 64], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv2_z = tf.nn.conv2d(conv1_a, filter=conv2_w, strides=strides, padding='SAME')
conv2_a = tf.nn.relu(conv2_z)

conv3_w = tf.get_variable('c3w', shape=[4, 4, 64, 128], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv3_z = tf.nn.conv2d(conv2_a, filter=conv3_w, strides=strides, padding='SAME')
conv3_a = tf.nn.relu(conv3_z)

conv4_w = tf.get_variable('c4w', shape=[4, 4, 128, 512], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
conv4_z = tf.nn.conv2d(conv3_a, filter=conv4_w, strides=strides, padding='VALID')
conv4_a = tf.nn.relu(conv4_z)

fc_in = tf.contrib.layers.flatten(conv4_a)
fc_w = tf.get_variable('fcw', shape=[n_hidden_units, fc_in.shape.as_list()[1]], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
fc_b = tf.Variable(np.zeros((n_hidden_units, 1)), dtype=tf.float32)
fc_z = tf.add(tf.matmul(fc_w, tf.transpose(fc_in)), fc_b)
fc_a = tf.nn.relu(fc_z)

w_out = tf.get_variable('w_out', shape=[1, n_hidden_units], dtype=tf.float32, initializer=tf.contrib.layers.xavier_initializer(seed=1337))
b_out = tf.Variable(np.zeros((1, 1)), dtype=tf.float32)
pred = tf.add(tf.matmul(w_out, fc_a), b_out)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

Instructions for updating:
Use keras.layers.flatten instead.
Instructions for updating:
Please use `layer.__call__` method instead.


In [6]:
cost = tf.reduce_mean(tf.squared_difference(pred, Y))

In [7]:
optimizer = tf.train.MomentumOptimizer(learning_rate, momentum=0.9).minimize(cost)

In [8]:
init = tf.global_variables_initializer()

In [9]:
batch = Batcher(n_examples, batch_size)
cost_train_batch = Batcher(n_examples, 10000)
cost_val_batch = Batcher(100000, 10000)

In [10]:
costs = []

with tf.Session() as sess:
    sess.run(init)
    
    for iteration in range(n_iterations):
        x_batch, y_batch = batch.next_batch([X_train, y_train])
        
        if iteration % display_step == 0:
            sys.stdout.write('*')
            x_batch_c, y_batch_c = cost_train_batch.next_batch([X_train, y_train])
            x_batch_v, y_batch_v = cost_val_batch.next_batch([X_val, y_val])
            c = sess.run(cost, feed_dict={X: x_batch_c, Y: y_batch_c.T})
            costs.append(c)
            pred_train = sess.run(pred, feed_dict={X: x_batch_c, Y: y_batch_c.T})
            pred_val = sess.run(pred, feed_dict={X: x_batch_v, Y: y_batch_v.T})
            print('it={} cost={}'.format(iteration, c))
            print(acc012(y_batch_c, pred_train.T))
            print(acc012(y_batch_v, pred_val.T))
            
        sess.run(optimizer, feed_dict={X:x_batch, Y:y_batch.T})
    
    # Testing
    graph = tf.get_default_graph()
    
    X_test = np.load('../../data/bin/test/deal.npy')
    y_test = np.load('../../data/bin/test/tricks_notrump.npy')
    
    print('\n\nTesting')
    predictions = sess.run(pred, feed_dict = {X:X_test})

    print(acc012(y_test.ravel(), predictions.round().ravel()))

*it=0 cost=43.30069351196289
(0.0177, 0.0542, 0.1125)
(0.0169, 0.0516, 0.1139)
*it=100000 cost=0.811424732208252
(0.4677, 0.908, 0.9876)
(0.461, 0.9131, 0.9882)
*it=200000 cost=0.6550103425979614
(0.513, 0.933, 0.9911)
(0.5141, 0.9321, 0.9917)
*it=300000 cost=0.5267388820648193
(0.5689, 0.9562, 0.994)
(0.5676, 0.9521, 0.9939)
*it=400000 cost=0.4866940379142761
(0.5914, 0.9608, 0.996)
(0.5854, 0.9587, 0.9958)
*it=500000 cost=0.42825058102607727
(0.6086, 0.9698, 0.9974)
(0.6064, 0.966, 0.9962)
*it=600000 cost=0.40252405405044556
(0.6211, 0.9731, 0.9977)
(0.6109, 0.9711, 0.9971)
*it=700000 cost=0.39047911763191223
(0.6267, 0.9719, 0.9983)
(0.6223, 0.97, 0.996)
*it=800000 cost=0.3605946898460388
(0.6497, 0.9784, 0.9979)
(0.6454, 0.9776, 0.9969)
*it=900000 cost=0.3551674783229828
(0.6568, 0.9775, 0.998)
(0.6479, 0.9744, 0.9979)
*it=1000000 cost=0.3315379321575165
(0.6652, 0.9816, 0.9984)
(0.6629, 0.9761, 0.9982)
*it=1100000 cost=0.3272269368171692
(0.667, 0.982, 0.9987)
(0.6609, 0.9781, 0.9

In [11]:
import pandas as pd
path = '../../bridgent/preds/z_v1_preds.pickle'
pd.to_pickle({'preds':predictions, 'acc012':acc012(y_test.ravel(), predictions.round().ravel())}, path)

In [12]:
pd.read_pickle(path)

{'preds': array([[5.1837964, 3.212062 , 7.1013417, ..., 5.3013005, 6.2725463,
         7.7386165]], dtype=float32),
 'acc012': (0.68139, 0.98248, 0.99831)}